In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Iniciando df

In [5]:
#importando dados

df_20 = pd.read_csv('state_of_data_2020.csv', low_memory=False)
df_21 = pd.read_csv('state_of_data_2021.csv', low_memory=False)
df_22 = pd.read_csv('State_of_data_2022.csv', low_memory=False)
df_23 = pd.read_csv('State_of_data_2023.csv', low_memory=False)
df_24 = pd.read_csv('State_of_data_2024.csv', low_memory=False)

In [ ]:
df_21["('P2', 'gender')"].value_counts()

('P2', 'gender')
Masculino    1436
Feminino      326
Name: count, dtype: int64

In [3]:
for col in df_20.columns:
   print(col)

('P0', 'id')
('P1', 'age')
('P2', 'gender')
('P3', 'living_in_brasil')
('P5', 'living_state')
('P6', 'born_or_graduated')
('P8', 'degreee_level')
('P10', 'job_situation')
('P12', 'workers_number')
('P13', 'manager')
('P16', 'salary_range')
('P17', 'time_experience_data_science')
('P18', 'time_experience_before')
('P19', 'is_data_science_professional')
('P20', 'linear_regression')
('P20', 'logistic_regression')
('P20', 'glms')
('P20', 'decision_tree')
('P20', 'random_forest')
('P20', 'neural_networks')
('P20', 'bayesian_inference')
('P20', 'ensemble')
('P20', 'svms')
('P20', 'cnns')
('P20', 'rnns')
('P20', 'hmms')
('P20', 'gans')
('P20', 'markov_chains')
('P20', 'nlp')
('P20', 'gradient_boosted_machines')
('P20', 'cluster_analysis')
('P20', 'survival_analysis')
('P20', 'longitudinal_data_analysis')
('P20', 'joint analysis')
('P20', 'no_listed_methods')
('P21', 'sql_')
('P21', 'r')
('P21', 'python')
('P21', 'c_c++_c#')
('P21', 'dotnet')
('P21', 'java')
('P21', 'julia')
('P21', 'sas_stata

# Preparando dados: padronizar nomes das colunas, selecionar apenas as relevantes e juntar os 3 anos em um único df

In [48]:
def load_data(dfs):
    """
    Carrega e padroniza os datasets do State of Data e padroniza as colunas relacionadas a gênero.
    
    Args:
        dfs: dicionário com ano como chave e DataFrame como valor
        
    Returns:
        DataFrame unificado com os dados dos três anos
    """
    datasets = dfs
    
    gender_columns = {
        '2021': "('P1_b ', 'Genero')",
        '2022': "('P1_b ', 'Genero')",
        '2023': "('P1_b ', 'Genero')",
        '2024': "('1.b_genero')"
    }
    
    column_mapping = {
        # Dados demográficos - Categoria P1
        '2021': {
            'idade': "('P1_a ', 'Idade')",
            'faixa_idade': "('P1_a_a ', 'Faixa idade')",
            'genero': "('P1_b ', 'Genero')",
            'estado': "('P1_e ', 'Estado onde mora')",
            'uf': "('P1_e_a ', 'uf onde mora')",
            'regiao': "('P1_e_b ', 'Regiao onde mora')",
            'regiao_origem': "('P1_g_b ', 'Regiao de origem')",
            'mudou_estado': "('P1_g_c ', 'Mudou de Estado?')",
            'nivel_ensino': "('P1_h ', 'Nivel de Ensino')",
            'area_formacao': "('P1_i ', 'Área de Formação')"
        },
        '2022': {
            'idade': "('P1_a ', 'Idade')",
            'faixa_idade': "('P1_a_1 ', 'Faixa idade')",
            'genero': "('P1_b ', 'Genero')",
            'estado': "('P1_i ', 'Estado onde mora')",
            'uf': "('P1_i_1 ', 'uf onde mora')",
            'regiao': "('P1_i_2 ', 'Regiao onde mora')",
            'regiao_origem': "('P1_k ', 'Regiao de origem')",
            'mudou_estado': "('P1_j ', 'Mudou de Estado?')",
            'nivel_ensino': "('P1_l ', 'Nivel de Ensino')",
            'area_formacao': "('P1_m ', 'Área de Formação')"
        },
        '2023': {
            'idade': "('P1_a ', 'Idade')",
            'faixa_idade': "('P1_a_1 ', 'Faixa idade')",
            'genero': "('P1_b ', 'Genero')",
            'estado': "('P1_i ', 'Estado onde mora')",
            'uf': "('P1_i_1 ', 'uf onde mora')",
            'regiao': "('P1_i_2 ', 'Regiao onde mora')",
            'regiao_origem': "('P1_k ', 'Regiao de origem')",
            'mudou_estado': "('P1_j ', 'Mudou de Estado?')",
            'nivel_ensino': "('P1_l ', 'Nivel de Ensino')",
            'area_formacao': "('P1_m ', 'Área de Formação')"
        },
        '2024': {
            'idade': '1.a_idade',
            'faixa_idade': '1.a.1_faixa_idade',
            'genero': '1.b_genero',
            'estado': '1.i_estado_onde_mora',
            'uf': '1.i.1_uf_onde_mora',
            'regiao': '1.i.2_regiao_onde_mora',
            'regiao_origem': '1.k.2_regiao_de_origem',
            'mudou_estado': '1.j_vive_no_estado_de_formacao',
            'nivel_ensino': '1.l_nivel_de_ensino',
            'area_formacao': '1.m_área_de_formação'
        }
    }
    
    # Colunas para informações profissionais 
    professional_columns = {
    '2021': {
        'situacao': "('P2_a ', 'Qual sua situação atual de trabalho?')",
        'setor': "('P2_b ', 'Setor')",
        'cargo': "('P2_f ', 'Cargo Atual')",
        'gestor':"('P2_d ', 'Gestor?')",
        'nivel': "('P2_g ', 'Nivel')",
        'faixa_salarial': "('P2_h ', 'Faixa salarial')",
        'tempo_area_dados': "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')",
        'tempo_area_ti': "('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')",
        'modalidade': "('P2_q ', 'Atualmente qual a sua forma de trabalho?')"
    },
    '2022': {
        'situacao': "('P2_a ', 'Qual sua situação atual de trabalho?')",
        'setor': "('P2_b ', 'Setor')",
        'cargo': "('P2_f ', 'Cargo Atual')",
        'gestor':"('P2_d ', 'Gestor?')",
        'nivel': "('P2_g ', 'Nivel')",
        'faixa_salarial': "('P2_h ', 'Faixa salarial')",
        'tempo_area_dados': "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')",
        'tempo_area_ti': "('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')",
        'modalidade': "('P2_p ', 'Atualmente qual a sua forma de trabalho?')"
    },
    '2023': {
        'situacao': "('P2_a ', 'Qual sua situação atual de trabalho?')",
        'setor': "('P2_b ', 'Setor')",
        'cargo': "('P2_f ', 'Cargo Atual')",
        'gestor':"('P2_d ', 'Gestor?')",
        'nivel': "('P2_g ', 'Nivel')",
        'faixa_salarial': "('P2_h ', 'Faixa salarial')",
        'tempo_area_dados': "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')",
        'tempo_area_ti': "('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')",
        'modalidade': "('P2_r ', 'Atualmente qual a sua forma de trabalho?')"
    },
    '2024': {
        'situacao': '2.a_situação_de_trabalho',
        'setor': '2.b_setor',
        'cargo': '2.f_cargo_atual',
        'gestor':'2.d_atua_como_gestor',
        'nivel': '2.g_nivel',
        'faixa_salarial': '2.h_faixa_salarial',
        'tempo_area_dados': '2.i_tempo_de_experiencia_em_dados',
        'tempo_area_ti': '2.j_tempo_de_experiencia_em_ti',
        'modalidade': '2.r_modelo_de_trabalho_atual'
    }
}
    
    
    processed_data = []
    
    for year, df in datasets.items():
        
        df['ano_pesquisa'] = year
        
        # Criando um novo DataFrame apenas com colunas relevantes
        year_data = pd.DataFrame()
        year_data['ano_pesquisa'] = df['ano_pesquisa']
        
        # Colunas demográficas
        for new_col, old_col in column_mapping[year].items():
            year_data[new_col] = df[old_col]
        
        # Colunas profissionais
        for new_col, old_col in professional_columns[year].items():
            year_data[new_col] = df[old_col]
        
        processed_data.append(year_data)
 
    combined_data = pd.concat(processed_data, ignore_index=True)
    
    return combined_data

dfs = {
    '2021': df_21,
    '2022': df_22,
    '2023': df_23,
    '2024': df_24
}

df_unificado = load_data(dfs)


In [49]:
df_unificado.head()

,ano_pesquisa,idade,faixa_idade,genero,estado,uf,regiao,regiao_origem,mudou_estado,nivel_ensino,area_formacao,situacao,setor,cargo,gestor,nivel,faixa_salarial,tempo_area_dados,tempo_area_ti,modalidade
0,2021,38.0,35-39,Masculino,Ceará (CE),CE,Nordeste,NaN,0,Pós-graduação,Química / Física,Empregado (CLT),Marketing,NaN,1.0,NaN,de R$ 4.001/mês a R$ 6.000/mês,Mais de 10 anos,Não tive experiência na área de TI/Engenharia ...,Modelo 100% presencial
1,2021,39.0,35-39,Masculino,Bahia (BA),BA,Nordeste,Sudeste,1,Pós-graduação,Economia/ Administração / Contabilidade / Fina...,Empreendedor ou Empregado (CNPJ),Consultoria,NaN,1.0,NaN,de R$ 6.001/mês a R$ 8.000/mês,de 2 a 3 anos,Não tive experiência na área de TI/Engenharia ...,Modelo híbrido flexível (o funcionário tem lib...
2,2021,30.0,30-34,Masculino,Santa Catarina (SC),SC,Sul,NaN,0,Pós-graduação,Computação / Engenharia de Software / Sistemas...,Empregado (CLT),Indústria,NaN,1.0,NaN,de R$ 8.001/mês a R$ 12.000/mês,de 1 a 2 anos,de 2 a 3 anos,Modelo 100% presencial
3,2021,38.0,35-39,Feminino,São Paulo (SP),SP,Sudeste,NaN,0,Pós-graduação,Outras,Empreendedor ou Empregado (CNPJ),Serviços,NaN,1.0,NaN,de R$ 12.001/mês a R$ 16.000/mês,Mais de 10 anos,Menos de 1 ano,Modelo 100% presencial
4,2021,36.0,35-39,Masculino,Santa Catarina (SC),SC,Sul,NaN,0,Pós-graduação,Outras Engenharias,Empreendedor ou Empregado (CNPJ),Varejo,NaN,1.0,NaN,de R$ 8.001/mês a R$ 12.000/mês,de 1 a 2 anos,de 4 a 5 anos,Modelo 100% presencial


In [50]:
df_rj = df_unificado[df_unificado['uf'] == 'RJ']

In [51]:
df_rj.to_excel("df_rj.xlsx", index=False)

In [52]:
df_rj["ano_pesquisa"].value_counts().sort_index()

ano_pesquisa
2021    222
2022    338
2023    437
2024    397
Name: count, dtype: int64

In [53]:
# ajustando coluna genero

df_rj = df_rj[df_rj['genero'].isin(['Masculino', 'Feminino'])]


In [54]:
df_rj.groupby("ano_pesquisa")["genero"].value_counts()

ano_pesquisa  genero   
2021          Masculino    178
              Feminino      44
2022          Masculino    260
              Feminino      77
2023          Masculino    327
              Feminino     109
2024          Masculino    293
              Feminino     101
Name: count, dtype: int64

In [55]:
# ajustando nivel ensino 

df_rj.loc[df_rj["nivel_ensino"] == 'Prefiro não informar', "nivel_ensino"] = np.nan

df_rj.loc[df_rj["nivel_ensino"] == 'Pós-graduação', "nivel_ensino"] = 'Especialização Lato Sensu'

ordem_nivel_ensino = ['Não tenho graduação formal', 'Estudante de Graduação', 
                      'Graduação/Bacharelado', 'Especialização Lato Sensu', 
                      'Mestrado', 'Doutorado ou Phd']

df_rj['nivel_ensino'] = pd.Categorical(df_rj['nivel_ensino'], categories=ordem_nivel_ensino, ordered=True)

In [56]:
df_rj.groupby("ano_pesquisa")["nivel"].value_counts()

ano_pesquisa  nivel 
2021          Pleno      66
              Júnior     47
              Sênior     40
2022          Júnior     84
              Sênior     75
              Pleno      72
2023          Sênior    122
              Pleno     117
              Júnior     93
2024          Sênior    112
              Pleno      95
              Júnior     80
Name: count, dtype: int64

In [57]:
# ajustando faixa salarial

df_rj["faixa_salarial"] = df_rj["faixa_salarial"].str.replace(r"\s+", " ", regex=True).str.strip()

df_rj["faixa_salarial"] = df_rj["faixa_salarial"].replace({
    'de R$ 1.001/mês a R$ 2.000/mês': "R$1k-2k",
    'de R$ 2.001/mês a R$ 3.000/mês': "R$2k-3k",
    'de R$ 2.001/mês a R$ 3000/mês': "R$2k-3k",
    'de R$ 3.001/mês a R$ 4.000/mês': "R$3k-4k", 
    'de R$ 4.001/mês a R$ 6.000/mês': "R$4k-6k",
    'de R$ 6.001/mês a R$ 8.000/mês': "R$6k-8k",
    'de R$ 8.001/mês a R$ 12.000/mês': "R$8k-12k",
    'de R$ 12.001/mês a R$ 16.000/mês': "R$12k-16k",
    'de R$ 16.001/mês a R$ 20.000/mês': "R$16k-20k",
    'de R$ 20.001/mês a R$ 25.000/mês': "R$20k-25k",
    'de R$ 25.001/mês a R$ 30.000/mês': "R$25k-30k",
    'de R$ 30.001/mês a R$ 40.000/mês': "R$30k-40k",
    'Acima de R$ 40.001/mês': "R$40k"
})

faixas_salariais_ordem = [ 
    'R$1k-2k',
    'R$2k-3k',
    'R$3k-4k', 
    'R$4k-6k',
    'R$6k-8k',
    'R$8k-12k',
    'R$12k-16k',
    'R$16k-20k',
    'R$20k-25k',
    'R$25k-30k',
    'R$30k-40k',
    'R$40k'
]

df_rj['faixa_salarial'] = pd.Categorical(df_rj['faixa_salarial'], categories=faixas_salariais_ordem, ordered=True)


In [58]:
df_rj.groupby("ano_pesquisa")["faixa_salarial"].value_counts()

ano_pesquisa  faixa_salarial
2021          R$8k-12k          46
              R$4k-6k           36
              R$6k-8k           28
              R$2k-3k           18
              R$12k-16k         18
              R$3k-4k           12
              R$1k-2k           10
              R$16k-20k         10
              R$20k-25k          4
              R$25k-30k          2
              R$30k-40k          1
              R$40k              1
2022          R$8k-12k          63
              R$12k-16k         44
              R$4k-6k           41
              R$3k-4k           33
              R$6k-8k           29
              R$2k-3k           23
              R$1k-2k           19
              R$16k-20k         14
              R$25k-30k          7
              R$20k-25k          5
              R$30k-40k          3
              R$40k              0
2023          R$8k-12k          95
              R$4k-6k           58
              R$6k-8k           48
              R$12k-16k   

In [59]:
# ajustando cargos

mapeamento = {
    # Analista de Dados
    'Analista de Dados/Data Analyst': 'Analista de Dados',
    'Analista de BI/BI Analyst': 'Analista de Dados',
    'Analista de BI/BI Analyst/Analytics Engineer': 'Analista de Dados',
    'Analista de Negócios/Business Analyst': 'Analista de Dados',
    'Analista de Inteligência de Mercado/Market Intelligence': 'Analista de Dados',
    'Analista de Marketing': 'Analista de Dados',
    'Analista Administrativo': 'Analista de Dados',
    'Estatístico': 'Analista de Dados',
    'Economista': 'Analista de Dados',
    
    # Engenheiro de Dados
    'Engenheiro de Dados/Arquiteto de Dados/Data Engineer/Data Architect': 'Engenheiro de Dados',
    'Engenheiro de Dados/Data Engineer': 'Engenheiro de Dados',
    'Analytics Engineer': 'Engenheiro de Dados',
    'Arquiteto de Dados': 'Engenheiro de Dados',
    'Arquiteto de dados': 'Engenheiro de Dados',
    'DBA/Administrador de Banco de Dados': 'Engenheiro de Dados',
    
    # Cientista de Dados
    'Cientista de Dados/Data Scientist': 'Cientista de Dados',
    'Engenheiro de Machine Learning/ML Engineer': 'Cientista de Dados',
    'Engenheiro de Machine Learning/ML Engineer/AI Engineer': 'Cientista de Dados',
    
    # Professor/Pesquisador
    'Professor': 'Professor/Pesquisador',
    'Professor/Pesquisador': 'Professor/Pesquisador',
    
    # Product Manager
    'Data Product Manager/ Product Manager (PM/APM/DPM/GPM/PO)': 'Product Manager',
    'Product Manager/ Product Owner (PM/APM/DPM/GPM/PO)': 'Product Manager',
    'Product Manager': 'Product Manager',
    
    # Desenvolvedor
    'Desenvolvedor/ Engenheiro de Software/ Analista de Sistemas': 'Desenvolvedor',
    'Desenvolvedor ou Engenheiro de Software': 'Desenvolvedor',
    'Analista de Sistemas/Analista de TI': 'Desenvolvedor',
    'Analista de Suporte/Analista Técnico': 'Desenvolvedor',
    'Suporte Técnico': 'Desenvolvedor',
    'Técnico': 'Desenvolvedor',
    
    # Categorias que não se encaixam exatamente
    'Outra Opção': 'Outro',
    'Outro': 'Outro',
    'Outras Engenharias (não inclui dev)': 'Outro'
}

df_rj['cargo'] = df_rj['cargo'].map(mapeamento).fillna(df_rj['cargo'])

In [60]:
df_rj.groupby("ano_pesquisa")["cargo"].value_counts()

ano_pesquisa  cargo                                           
2021          Analista de Dados                                    84
              Cientista de Dados                                   34
              Engenheiro de Dados                                  17
              Desenvolvedor                                         9
              Outro                                                 9
2022          Analista de Dados                                    93
              Cientista de Dados                                   53
              Engenheiro de Dados                                  43
              Outro                                                28
              Desenvolvedor                                        13
              Professor/Pesquisador                                 1
2023          Analista de Dados                                   151
              Engenheiro de Dados                                  69
              Cientista de 

In [61]:
# ajustando nivel senioridade

df_rj["nivel"] = df_rj["nivel"].astype('category')

if 'Gestor' not in df_rj["nivel"].cat.categories:
    df_rj["nivel"] = df_rj["nivel"].cat.add_categories(['Gestor'])


df_rj.loc[df_rj["gestor"] == 1.0, "nivel"] = "Gestor"

df_rj["nivel"] = df_rj["nivel"].cat.reorder_categories(['Júnior', 'Pleno', 'Sênior', 'Gestor'])


In [62]:
# ajustando áreas de formação

mapeamento_areas = {
    "Química / Física": "Química / Física",
    "Economia/ Administração / Contabilidade / Finanças": "Economia / Administração / Finanças / Negócios",
    "Computação / Engenharia de Software / Sistemas de Informação/ TI": "Computação / Engenharia de Software / TI",
    "Outras": "Outras",
    "Outras Engenharias": "Engenharia (outras)",
    "Marketing / Publicidade / Comunicação / Jornalismo": "Marketing / Comunicação / Jornalismo",
    "Estatística/ Matemática / Matemática Computacional": "Estatística / Matemática / Ciências Atuariais",
    "Ciências Sociais": "Ciências Sociais",
    "Ciências Biológicas/Farmácia/Medicina/Área da Saúde": "Ciências Biológicas / Medicina / Saúde",
    "Estatística/ Matemática / Matemática Computacional/ Ciências Atuariais": "Estatística / Matemática / Ciências Atuariais",
    "Ciências Biológicas/ Farmácia/ Medicina/ Área da Saúde": "Ciências Biológicas / Medicina / Saúde",
    "Economia/ Administração / Contabilidade / Finanças/ Negócios": "Economia / Administração / Finanças / Negócios",
    "Outra opção": "Outras"
}

df_rj["area_formacao"] = df_rj["area_formacao"].replace(mapeamento_areas)

In [63]:
df_rj["setor"].value_counts()

setor
Tecnologia/Fábrica de Software         222
Finanças ou Bancos                     133
Varejo                                 101
Área de Consultoria                     99
Educação                                95
Outra Opção                             77
Área da Saúde                           66
Telecomunicação                         54
Setor de Energia                        47
Setor Público                           46
Indústria                               42
Internet/Ecommerce                      40
Marketing                               36
Entretenimento ou Esportes              35
Seguros ou Previdência                  35
Setor Alimentício                       30
Outro                                   12
Setor Imobiliário/ Construção Civil     11
Agronegócios                             9
Consultoria                              7
Filantropia/ONG's                        7
Energia                                  7
Setor Farmaceutico                       6
Logís

In [64]:
df_rj.head()

,ano_pesquisa,idade,faixa_idade,genero,estado,uf,regiao,regiao_origem,mudou_estado,nivel_ensino,area_formacao,situacao,setor,cargo,gestor,nivel,faixa_salarial,tempo_area_dados,tempo_area_ti,modalidade
53,2021,39.0,35-39,Masculino,Rio de Janeiro (RJ),RJ,Sudeste,NaN,0,Não tenho graduação formal,NaN,Empregado (CLT),Varejo,NaN,1.0,Gestor,R$8k-12k,de 6 a 10 anos,Não tive experiência na área de TI/Engenharia ...,Modelo 100% remoto
54,2021,33.0,30-34,Masculino,Rio de Janeiro (RJ),RJ,Sudeste,NaN,0,Especialização Lato Sensu,Computação / Engenharia de Software / TI,Empregado (CLT),Varejo,NaN,1.0,Gestor,R$8k-12k,Mais de 10 anos,de 1 a 2 anos,Modelo híbrido com dias fixos de trabalho pres...
57,2021,37.0,35-39,Masculino,Rio de Janeiro (RJ),RJ,Sudeste,NaN,0,Especialização Lato Sensu,Computação / Engenharia de Software / TI,Empregado (CLT),Marketing,NaN,1.0,Gestor,R$20k-25k,de 6 a 10 anos,Mais de 10 anos,Modelo 100% remoto
85,2021,32.0,30-34,Masculino,Rio de Janeiro (RJ),RJ,Sudeste,Sudeste,1,Especialização Lato Sensu,Economia / Administração / Finanças / Negócios,Servidor Público,Setor Público,NaN,1.0,Gestor,R$8k-12k,Não tenho experiência na área de dados,Não tive experiência na área de TI/Engenharia ...,Modelo 100% presencial
88,2021,35.0,35-39,Feminino,Rio de Janeiro (RJ),RJ,Sudeste,NaN,0,Especialização Lato Sensu,Economia / Administração / Finanças / Negócios,Empregado (CLT),Internet/Ecommerce,NaN,1.0,Gestor,R$20k-25k,de 6 a 10 anos,Não tive experiência na área de TI/Engenharia ...,Modelo 100% remoto


# Visualizando os dados

In [65]:
# Distribuição faixa salarial x genero

def criar_piramide_salarial(df_rj):
    # Configurações de cores
    cores = {"Masculino": "#708090", "Feminino": "#673147"}

    # Criando figura com subplots
    fig = make_subplots(
        rows=1,
        cols=4,
        subplot_titles=("2021", "2022", "2023", "2024"),
        shared_yaxes=True
    )

    anos = ['2021', '2022', '2023', '2024']

    for idx, ano in enumerate(anos, 1):
        # Filtrando dados do ano atual
        df_grouped = df_rj[df_rj["ano_pesquisa"] == ano]
        df_grouped = df_grouped.groupby(["faixa_salarial", "genero"]).size().reset_index(name="count")
        
        # Pivot para separar por gênero
        df_plot = df_grouped.pivot(index="faixa_salarial", columns="genero", values="count").fillna(0)

        # Garantindo que as colunas existam
        for genero in ["Masculino", "Feminino"]:
            if genero not in df_plot.columns:
                df_plot[genero] = 0

        # Adicionando barras para Feminino
        fig.add_trace(
            go.Bar(
                y=df_plot.index,
                x=df_plot["Feminino"],
                name="Feminino",  # Removemos o ano para agrupar na legenda
                orientation="h",
                marker=dict(color=cores["Feminino"]),
                text=df_plot["Feminino"].astype(int),
                textposition='auto',
                showlegend=True if idx == 1 else False  # Exibe legenda apenas no primeiro ano
            ),
            row=1,
            col=idx
        )

        # Adicionando barras para Masculino
        fig.add_trace(
            go.Bar(
                y=df_plot.index,
                x=df_plot["Masculino"],
                name="Masculino",  # Removemos o ano para agrupar na legenda
                orientation="h",
                marker=dict(color=cores["Masculino"]),
                text=df_plot["Masculino"].astype(int),
                textposition='auto',
                showlegend=True if idx == 1 else False  # Exibe legenda apenas no primeiro ano
            ),
            row=1,
            col=idx
        )

    # Configurando layout
    fig.update_layout(
        title="Distribuição Salarial por Gênero (2021-2023)",
        template="plotly_white",
        height=600,
        bargap=0.1,
        showlegend=True
    )

    # Configurando eixos
    fig.update_xaxes(title_text="Quantidade de pessoas")
    fig.update_yaxes(title_text="Faixa Salarial")

    return fig

# Usar a função
fig = criar_piramide_salarial(df_rj)
fig.show()

C:\Users\julli\AppData\Local\Temp\ipykernel_8168\1366915577.py:20: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\julli\AppData\Local\Temp\ipykernel_8168\1366915577.py:20: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\julli\AppData\Local\Temp\ipykernel_8168\1366915577.py:20: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\julli\AppData\Local\Temp\ipykernel_8168\1366915577.py:20: FutureWarning:

Th

In [66]:
# Proporção niveis senioridade x genero

# Criando um DataFrame agrupado para contar quantas pessoas há por nível de senioridade e gênero
df_grouped = df_rj.groupby(["ano_pesquisa", "nivel", "genero"]).size().reset_index(name="count")

# Calculando o total por ano e gênero para depois calcular proporções
totais_por_ano_genero = df_rj.groupby(["ano_pesquisa", "genero"]).size().reset_index(name="total")
df_grouped = df_grouped.merge(totais_por_ano_genero, on=["ano_pesquisa", "genero"])

# Calculando a proporção
df_grouped["proporção"] = df_grouped["count"] / df_grouped["total"] * 100

# Criando a figura com subplots
fig = make_subplots(rows=1, cols=4, shared_yaxes=True, subplot_titles=["2021", "2022", "2023", "2024"])

anos = ['2021', '2022', '2023', '2024']
cores = {"Masculino": "#708090", "Feminino": "#673147"}

for i, ano in enumerate(anos):
    # Filtrando dados do ano atual
    df_ano = df_grouped[df_grouped["ano_pesquisa"] == ano]
    
    # Criando um pivot para senioridade x gênero
    df_plot = df_ano.pivot(index="nivel", columns="genero", values="proporção").fillna(0)
    
    # Garantindo que as colunas existam, preenchendo com 0 se necessário
    for genero in ["Masculino", "Feminino"]:
        if genero not in df_plot.columns:
            df_plot[genero] = 0
    
    # Filtrando níveis que têm pelo menos um registro
    df_plot = df_plot[(df_plot["Masculino"] > 0) | (df_plot["Feminino"] > 0)]
    
    # Se não houver dados, continue para o próximo ano
    if df_plot.empty:
        continue
    
    # Adicionando barras para Feminino (negativo para inverter no gráfico)
    fig.add_trace(go.Bar(
        y=df_plot.index,
        x=-df_plot["Feminino"],
        name="Feminino",
        orientation="h",
        marker=dict(color=cores["Feminino"]),
        text=[f"{val:.1f}%" for val in df_plot["Feminino"]],
        textposition="outside",
        showlegend=True if i == 0 else False
    ), row=1, col=i+1)

    # Adicionando barras para Masculino
    fig.add_trace(go.Bar(
        y=df_plot.index,
        x=df_plot["Masculino"],
        name="Masculino",
        orientation="h",
        marker=dict(color=cores["Masculino"]),
        text=[f"{val:.1f}%" for val in df_plot["Masculino"]],
        textposition="outside",
        showlegend=True if i == 0 else False
    ), row=1, col=i+1)

# Configurando layout
fig.update_layout(
    title="Distribuição de Senioridade por Gênero (2021-2024)",
    xaxis_title="Proporção (%)",
    yaxis_title="Nível de Senioridade",
    barmode="relative",
    template="plotly_white"
)

# Ajustando os eixos X para mostrar valores em percentuais
for i in range(1, 5):
    fig.update_xaxes(ticksuffix="%", range=[-100, 100], row=1, col=i)
    
    # Adicionando linhas de referência em 0
    fig.add_shape(
        type="line",
        x0=0, y0=-0.5,
        x1=0, y1=len(df_plot.index) - 0.5,
        line=dict(color="black", width=1, dash="dot"),
        row=1, col=i
    )

fig.show()

C:\Users\julli\AppData\Local\Temp\ipykernel_8168\3022373491.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [67]:

niveis_desejados = ["Júnior", "Pleno", "Sênior", "Gestor"] 

# Normaliza os nomes dos níveis para minúsculas no filtro
df_filtrado = df_rj[df_rj["nivel"].str.lower().isin([n.lower() for n in niveis_desejados])]

# Agora agrupe pelos campos desejados
df_grouped = df_filtrado.groupby(["ano_pesquisa", "nivel", "genero"]).size().reset_index(name="count")

# Calculando o total por ano e gênero para depois calcular proporções
totais_por_ano_genero = df_filtrado.groupby(["ano_pesquisa", "genero"]).size().reset_index(name="total")
df_grouped = df_grouped.merge(totais_por_ano_genero, on=["ano_pesquisa", "genero"])

# Calculando a proporção
df_grouped["proporção"] = df_grouped["count"] / df_grouped["total"] * 100

# Verificação (opcional): soma das proporções por ano e gênero deve ser 100%
verificacao = df_grouped.groupby(["ano_pesquisa", "genero"])["proporção"].sum().reset_index()
print("Verificação das somas por gênero:")
print(verificacao)

fig = make_subplots(rows=1, cols=4, shared_yaxes=True, subplot_titles=["2021", "2022", "2023", "2024"])

anos = ['2021', '2022', '2023', '2024']
cores = {"Masculino": "#708090", "Feminino": "#673147"}

for i, ano in enumerate(anos):
    # Filtrando dados do ano atual
    df_ano = df_grouped[df_grouped["ano_pesquisa"] == ano]
    
    # Criando um pivot para senioridade x gênero
    df_plot = df_ano.pivot(index="nivel", columns="genero", values="proporção").fillna(0)
    
    # Garantindo que as colunas existam, preenchendo com 0 se necessário
    for genero in ["Masculino", "Feminino"]:
        if genero not in df_plot.columns:
            df_plot[genero] = 0
    
    # Filtrando níveis que têm pelo menos um registro
    df_plot = df_plot[(df_plot["Masculino"] > 0) | (df_plot["Feminino"] > 0)]
    
    # Se não houver dados, continue para o próximo ano
    if df_plot.empty:
        continue
    
    # Adicionando barras para Feminino (negativo para inverter no gráfico)
    # Arredondando as porcentagens para números inteiros
    fig.add_trace(go.Bar(
        y=df_plot.index,
        x=-df_plot["Feminino"],
        name="Feminino",
        orientation="h",
        marker=dict(color=cores["Feminino"]),
        text=[f"{int(round(val))}%" for val in df_plot["Feminino"]],  # Arredondamento para inteiros
        textposition="outside",
        showlegend=True if i == 0 else False
    ), row=1, col=i+1)

    # Adicionando barras para Masculino
    # Arredondando as porcentagens para números inteiros
    fig.add_trace(go.Bar(
        y=df_plot.index,
        x=df_plot["Masculino"],
        name="Masculino",
        orientation="h",
        marker=dict(color=cores["Masculino"]),
        text=[f"{int(round(val))}%" for val in df_plot["Masculino"]],  # Arredondamento para inteiros
        textposition="outside",
        showlegend=True if i == 0 else False
    ), row=1, col=i+1)

# Configurando layout
fig.update_layout(
    title="Distribuição de Senioridade por Gênero (2021-2024)",
    xaxis_title="Proporção (%)",
    yaxis_title="Nível de Senioridade",
    barmode="relative",
    template="plotly_white"
)

# Ajustando os eixos X para mostrar valores em percentuais
for i in range(1, 5):
    fig.update_xaxes(ticksuffix="%", range=[-100, 100], row=1, col=i)
    
    # Adicionando linhas de referência em 0
    fig.add_shape(
        type="line",
        x0=0, y0=-0.5,
        x1=0, y1=len(df_plot.index) - 0.5,
        line=dict(color="black", width=1, dash="dot"),
        row=1, col=i
    )

fig.show()

Verificação das somas por gênero:
  ano_pesquisa     genero  proporção
0         2021   Feminino      100.0
1         2021  Masculino      100.0
2         2022   Feminino      100.0
3         2022  Masculino      100.0
4         2023   Feminino      100.0
5         2023  Masculino      100.0
6         2024   Feminino      100.0
7         2024  Masculino      100.0


C:\Users\julli\AppData\Local\Temp\ipykernel_8168\1547542143.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [68]:
# Distribuição escolaridade x genero

df_grouped = df_rj.groupby(["ano_pesquisa", "nivel_ensino", "genero"]).size().reset_index(name="count")

totais_por_ano_nivel = df_rj.groupby(["ano_pesquisa", "nivel_ensino"]).size().reset_index(name="total_nivel")
df_grouped = df_grouped.merge(totais_por_ano_nivel, on=["ano_pesquisa", "nivel_ensino"])

df_grouped["proporção"] = df_grouped["count"] / df_grouped["total_nivel"] * 100

fig = make_subplots(rows=1, cols=4, shared_yaxes=True, subplot_titles=["2021", "2022", "2023", "2024"])

anos = ['2021', '2022', '2023', '2024']
cores = {"Masculino": "#708090", "Feminino": "#673147"}

ordem_niveis = [
    'Não tenho graduação formal', 
    'Estudante de Graduação',
    'Graduação/Bacharelado', 
    'Pós-graduação',
    'Mestrado',
    'Doutorado ou Phd'
]

for i, ano in enumerate(anos):
   
    df_ano = df_grouped[df_grouped["ano_pesquisa"] == ano]
    
    for nivel in ordem_niveis:
        df_nivel = df_ano[df_ano["nivel_ensino"] == nivel]
        
        if df_nivel.empty:
            continue
        
        
        proporcoes = {genero: df_nivel[df_nivel["genero"] == genero]["proporção"].values[0] 
                      if not df_nivel[df_nivel["genero"] == genero].empty else 0 
                      for genero in ["Masculino", "Feminino"]}
        
        
        for genero in ["Feminino", "Masculino"]:
            fig.add_trace(go.Bar(
                y=[nivel],
                x=[proporcoes[genero]],
                name=genero,
                orientation="h",
                marker=dict(color=cores[genero]),
                text=f"{proporcoes[genero]:.1f}%",
                textposition="inside",
                showlegend=True if i == 0 and nivel == ordem_niveis[0] else False
            ), row=1, col=i+1)

# Configurando layout
fig.update_layout(
    title="Distribuição por Gênero em Cada Nível de Formação Acadêmica (2021-2024)",
    xaxis_title="Proporção (%)",
    yaxis_title="Nível de Formação",
    barmode="stack",
    height=600,
    template="plotly_white",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)

# Ajustando os eixos X para mostrar valores em percentuais
for i in range(1, 5):
    fig.update_xaxes(ticksuffix="%", range=[0, 100], row=1, col=i)
    

fig.show()

C:\Users\julli\AppData\Local\Temp\ipykernel_8168\681661558.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\julli\AppData\Local\Temp\ipykernel_8168\681661558.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [69]:
# distribuição area formação x genero

df = df_rj[df_rj['area_formacao'] != "Não classificado"].copy()

df_contagem = df.groupby(['area_formacao', 'ano_pesquisa', 'genero']).size().reset_index(name='contagem')


cores = {"Feminino": "#673147", "Masculino": "#708090"}


areas_unicas = df_contagem.groupby('area_formacao')['contagem'].sum().sort_values(ascending=False).index.tolist()
anos = sorted(df_contagem['ano_pesquisa'].unique())
generos = ["Feminino", "Masculino"]


n_cols = 3  
n_rows = -(-len(areas_unicas) // n_cols)  


fig = make_subplots(
    rows=n_rows, 
    cols=n_cols,
    subplot_titles=areas_unicas,
    shared_xaxes=True,
    horizontal_spacing=0.07,
    vertical_spacing=0.12
)


for i, area in enumerate(areas_unicas):
    row = i // n_cols + 1
    col = i % n_cols + 1
    
    for genero in generos:
        df_filtrado = df_contagem[(df_contagem['area_formacao'] == area) & 
                                  (df_contagem['genero'] == genero)]
        
        x_anos = []
        y_contagens = []
        
        for ano in anos:
            df_ano = df_filtrado[df_filtrado['ano_pesquisa'] == ano]
            y_contagens.append(df_ano['contagem'].values[0] if not df_ano.empty else 0)
            x_anos.append(ano)
        
        fig.add_trace(
            go.Bar(
                x=x_anos,
                y=y_contagens,
                name=genero,
                marker_color=cores[genero],
                text=y_contagens,
                textposition='auto',
                showlegend=i == 0
            ),
            row=row, col=col
        )

fig.update_layout(
    title='Distribuição de Áreas de Formação por Ano e Gênero',
    height=min(1000, n_rows * 250),
    width=1200,
    template='plotly_white',
    barmode='group',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(t=80, b=50, l=50, r=50)
)


for i in range(1, n_rows + 1):
    for j in range(1, n_cols + 1):
        if (i-1) * n_cols + j <= len(areas_unicas):
            fig.update_xaxes(title_text="Ano" if i == n_rows else "", tickangle=45, row=i, col=j)
            if j == 1:
                fig.update_yaxes(title_text="Quantidade", row=i, col=j)

fig.show()


In [70]:
# Distribuição escolaridade x genero - VALORES ABSOLUTOS, BARRAS LADO A LADO, SEM REPETIR NOME DO GÊNERO NO EIXO Y

df_grouped = df_unificado.groupby(["ano_pesquisa", "nivel_ensino", "genero"]).size().reset_index(name="count")

fig = make_subplots(rows=1, cols=4, shared_yaxes=True, subplot_titles=["2021", "2022", "2023", "2024"])

anos = ['2021', '2022', '2023', '2024']
cores = {"Masculino": "#708090", "Feminino": "#673147"}

ordem_niveis = [
    'Não tenho graduação formal', 
    'Estudante de Graduação',
    'Graduação/Bacharelado', 
    'Especialização Lato Sensu',
    'Mestrado',
    'Doutorado ou Phd'
]

for i, ano in enumerate(anos):
    df_ano = df_grouped[df_grouped["ano_pesquisa"] == ano]
    
    for genero in ["Feminino", "Masculino"]:
        counts = []
        niveis = []
        
        for nivel in ordem_niveis:
            df_nivel = df_ano[(df_ano["nivel_ensino"] == nivel) & (df_ano["genero"] == genero)]
            count = df_nivel["count"].values[0] if not df_nivel.empty else 0
            counts.append(count)
            niveis.append(nivel)
        
        fig.add_trace(go.Bar(
            y=niveis,
            x=counts,
            name=genero,
            orientation="h",
            marker=dict(color=cores[genero]),
            text=[str(c) for c in counts],
            textposition="outside",
            textfont=dict(color="black"),
            showlegend=True if i == 0 else False
        ), row=1, col=i+1)

# Configurando layout
fig.update_layout(
    title="Distribuição Absoluta por Gênero em Cada Nível de Formação Acadêmica (2021-2023)",
    xaxis_title="Quantidade",
    yaxis_title="Nível de Formação",
    barmode="group",  # Barras lado a lado
    height=800,
    template="plotly_white",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    )
)

fig.show()


In [71]:
df_grouped = df_rj.groupby(["ano_pesquisa", "nivel_ensino", "genero"]).size().reset_index(name="count")

# Não precisamos mais calcular a proporção, mas podemos manter os totais por nível para referência
totais_por_ano_nivel = df_rj.groupby(["ano_pesquisa", "nivel_ensino"]).size().reset_index(name="total_nivel")
df_grouped = df_grouped.merge(totais_por_ano_nivel, on=["ano_pesquisa", "nivel_ensino"])

fig = make_subplots(rows=1, cols=4, shared_yaxes=True, subplot_titles=["2021", "2022", "2023", "2024"])

anos = ['2021', '2022', '2023', '2024']
cores = {"Masculino": "#708090", "Feminino": "#673147"}

ordem_niveis = [
    'Não tenho graduação formal', 
    'Estudante de Graduação',
    'Graduação/Bacharelado', 
    'Pós-graduação',
    'Mestrado',
    'Doutorado ou Phd'
]

# Definir uma largura fixa para as barras
largura_barra = 0.7

for i, ano in enumerate(anos):
   
    df_ano = df_grouped[df_grouped["ano_pesquisa"] == ano]
    
    for nivel in ordem_niveis:
        df_nivel = df_ano[df_ano["nivel_ensino"] == nivel]
        
        if df_nivel.empty:
            continue
        
        # Usando valores absolutos (count) em vez de proporção
        contagens = {genero: df_nivel[df_nivel["genero"] == genero]["count"].values[0] 
                      if not df_nivel[df_nivel["genero"] == genero].empty else 0 
                      for genero in ["Masculino", "Feminino"]}
        
        for genero in ["Feminino", "Masculino"]:
            fig.add_trace(go.Bar(
                y=[nivel],
                x=[contagens[genero]],
                name=genero,
                orientation="h",
                marker=dict(color=cores[genero]),
                text=f"{int(contagens[genero])}",  # Formatando como número inteiro
                textposition="inside",
                textangle=0,  # Garantindo que o texto esteja na horizontal
                width=largura_barra,  # Definindo uma largura fixa para todas as barras
                showlegend=True if i == 0 and nivel == ordem_niveis[0] else False
            ), row=1, col=i+1)

# Configurando layout
fig.update_layout(
    title="Distribuição por Gênero em Cada Nível de Formação Acadêmica (2021-2023)",
    xaxis_title="Contagem Absoluta",
    yaxis_title="Nível de Formação",
    barmode="stack",
    height=600,
    template="plotly_white",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    uniformtext_minsize=10,  # Tamanho mínimo do texto para garantir legibilidade
    uniformtext_mode='hide',  # Esconde o texto se não couber na barra
    bargap=0.2,  # Espaçamento entre grupos de barras
    bargroupgap=0.1  # Espaçamento entre barras do mesmo grupo
)

# Ajustando os eixos X e configurando uniformidade entre os subplots
max_value = df_grouped["count"].max() * 1.1  # 10% a mais para dar margem

for i in range(1, 5):
    fig.update_xaxes(
        row=1, 
        col=i, 
        range=[0, max_value],  # Definindo a mesma escala para todos os gráficos
        tickformat=",d"  # Formato para inteiros com separador de milhar
    )

# Definindo a mesma escala para o eixo Y em todos os subplots
fig.update_yaxes(
    categoryorder='array',
    categoryarray=ordem_niveis
)
    
fig.show()

C:\Users\julli\AppData\Local\Temp\ipykernel_8168\2918085246.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\julli\AppData\Local\Temp\ipykernel_8168\2918085246.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [72]:
df_grouped = df_rj.groupby(["ano_pesquisa", "nivel_ensino", "genero"]).size().reset_index(name="count")

fig = make_subplots(
    rows=1, 
    cols=4, 
    shared_yaxes=True, 
    subplot_titles=["2021", "2022", "2023", "2024"],
    horizontal_spacing=0.07
)

anos = ['2021', '2022', '2023', '2024']
cores = {"Masculino": "#708090", "Feminino": "#673147"}

ordem_niveis = [
    'Não tenho graduação formal', 
    'Estudante de Graduação',
    'Graduação/Bacharelado', 
    'Especialização Lato Sensu',
    'Mestrado',
    'Doutorado ou Phd'
]

# Para ajustar range máximo depois
max_count = df_grouped['count'].max()

for i, ano in enumerate(anos):
    df_ano = df_grouped[df_grouped["ano_pesquisa"] == ano]
    
    for genero in ["Feminino", "Masculino"]:
        counts = []
        niveis = []
        
        for nivel in ordem_niveis:
            df_nivel = df_ano[(df_ano["nivel_ensino"] == nivel) & (df_ano["genero"] == genero)]
            count = df_nivel["count"].values[0] if not df_nivel.empty else 0
            counts.append(count)
            niveis.append(nivel)
        
        fig.add_trace(go.Bar(
            y=niveis,
            x=counts,
            name=genero,
            orientation="h",
            marker=dict(color=cores[genero]),
            text=[str(c) for c in counts],
            textposition="outside",  # Deixa o número fora da barra
            textfont=dict(color="black", size=10),
            showlegend=True if i == 0 else False
        ), row=1, col=i+1)

# Configurando layout
fig.update_layout(
    title="Distribuição Absoluta por Gênero em Cada Nível de Formação Acadêmica (2021-2023)",
    title_font_size=16,
    barmode="group",
    height=800,
    template="plotly_white",
    font=dict(size=11),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
        font=dict(size=10)
    )
)

# Ajustando range de cada xaxis para dar espaço aos textos "outside"
for i in range(1, 5):
    fig.update_xaxes(row=1, col=i, range=[0, max_count * 1.2])  # Aumenta 20% do maior valor

fig.show()


C:\Users\julli\AppData\Local\Temp\ipykernel_8168\2750855949.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [73]:
# Distribuição cargo x genero

df = df_rj[df_rj['cargo'] != "Não classificado"].copy()

df_contagem = df.groupby(['cargo', 'ano_pesquisa', 'genero']).size().reset_index(name='contagem')


cores = {"Feminino": "#673147", "Masculino": "#708090"}

cargos_unicos = sorted(df_contagem['cargo'].unique())
anos = sorted(df_contagem['ano_pesquisa'].unique())
generos = ["Feminino", "Masculino"]


fig = make_subplots(
    rows=len(anos), 
    cols=1,
    subplot_titles=[f"Ano {ano}" for ano in anos], 
    shared_xaxes=True,
    vertical_spacing=0.05
)


for i, ano in enumerate(anos):
    row = i + 1  
    
    for genero in generos:
        
        df_filtrado = df_contagem[(df_contagem['ano_pesquisa'] == ano) & 
                                  (df_contagem['genero'] == genero)]
        
        
        y_cargos = []
        x_contagens = []
        
        for cargo in cargos_unicos:
            df_cargo = df_filtrado[df_filtrado['cargo'] == cargo]
            if not df_cargo.empty:
                x_contagens.append(df_cargo['contagem'].values[0])
            else:
                x_contagens.append(0)  #
            y_cargos.append(cargo)
        
        
        fig.add_trace(
            go.Bar(
                y=y_cargos,                # Cargos no eixo Y
                x=x_contagens,             # Contagens no eixo X
                name=genero,               # Nome na legenda
                marker_color=cores[genero],# Cor com base no gênero
                textposition='auto',
                orientation='h',           # Barras horizontais
                showlegend=i == 0          # Mostrar na legenda apenas para o primeiro ano
            ),
            row=row, col=1
        )

fig.update_layout(
    title='Distribuição de Cargos por Gênero ao Longo dos Anos',
    height=max(800, len(anos) * 200),  # Ajuste dinâmico da altura
    width=1000,
    template='plotly_white',
    barmode='group',              # Agrupar barras por gênero
    legend=dict(
        orientation="h",          # Legenda horizontal
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)


fig.update_xaxes(title_text="Quantidade", row=len(anos), col=1)


for i in range(1, len(anos) + 1):
    fig.update_yaxes(title_text="Cargo", row=i, col=1, showgrid=True)

fig.show()


# Estatísticas

In [74]:
percentuais_genero = df_rj.groupby('ano_pesquisa')['genero'].apply(
    lambda x: (x == 'Feminino').mean() * 100
).reset_index(name='percentual_mulheres')

print("Percentual de mulheres por ano:")
print(percentuais_genero)

ordem_nivel_ensino = ['Especialização Lato Sensu', 
                      'Mestrado', 'Doutorado ou Phd']
escolaridade_superior = df_rj.groupby('genero')['nivel_ensino'].apply(
    lambda x: (x.isin(ordem_nivel_ensino)).mean() * 100
).reset_index(name='percentual_pos_ou_superior')

print("Percentual com pós-graduação ou superior por gênero:")
print(escolaridade_superior)

# Percentual geral de mulheres
perc_total_mulheres = (df_rj['genero'] == 'Feminino').mean() * 100

# Filtragem para cargos de liderança - apenas cargo Gestor
cargos_lideranca = ['Gestor']
df_lideranca = df_rj[df_rj['nivel'].isin(cargos_lideranca)]
perc_mulheres_lideranca = (df_lideranca['genero'] == 'Feminino').mean() * 100

# Filtragem para níveis seniores técnicos
niveis_senior = ['Sênior']
df_senior = df_rj[df_rj['nivel'].isin(niveis_senior)]
perc_mulheres_senior = (df_senior['genero'] == 'Feminino').mean() * 100

# Filtragem para nível júnior
df_junior = df_rj[df_rj['nivel'] == 'Júnior']
perc_mulheres_junior = (df_junior['genero'] == 'Feminino').mean() * 100

print(f"Percentual geral de mulheres: {perc_total_mulheres:.2f}%")
print(f"Percentual de mulheres em cargos de liderança: {perc_mulheres_lideranca:.2f}%")
print(f"Percentual de mulheres em níveis seniores: {perc_mulheres_senior:.2f}%")
print(f"Percentual de mulheres em nível júnior: {perc_mulheres_junior:.2f}%")

Percentual de mulheres por ano:
  ano_pesquisa  percentual_mulheres
0         2021            19.819820
1         2022            22.848665
2         2023            25.000000
3         2024            25.634518
Percentual com pós-graduação ou superior por gênero:
      genero  percentual_pos_ou_superior
0   Feminino                   58.308157
1  Masculino                   48.298677
Percentual geral de mulheres: 23.83%
Percentual de mulheres em cargos de liderança: 25.45%
Percentual de mulheres em níveis seniores: 24.64%
Percentual de mulheres em nível júnior: 22.70%


In [75]:
# Percentual geral de mulheres por ano
perc_total_mulheres_por_ano = df_rj.groupby('ano_pesquisa')['genero'].apply(
    lambda x: (x == 'Feminino').mean() * 100
).reset_index(name='percentual_mulheres')

# Filtragem para cargos de liderança - apenas cargo Gestor, por ano
cargos_lideranca = ['Júnior']
df_lideranca = df_rj[df_rj['nivel'].isin(cargos_lideranca)]
perc_mulheres_lideranca_por_ano = df_lideranca.groupby('ano_pesquisa')['genero'].apply(
    lambda x: (x == 'Feminino').mean() * 100
).reset_index(name='percentual_mulheres_gestoras')

# Mostrar resultados
print("Percentual geral de mulheres por ano:")
print(perc_total_mulheres_por_ano)

print("\nPercentual de mulheres em cargos de gestão por ano:")
print(perc_mulheres_lideranca_por_ano)


for ano in df_rj['ano_pesquisa'].unique():
    total_mulheres = perc_total_mulheres_por_ano[perc_total_mulheres_por_ano['ano_pesquisa'] == ano]['percentual_mulheres'].values[0]
    
    # Verificar se existem gestoras neste ano antes de acessar
    gestoras_ano = perc_mulheres_lideranca_por_ano[perc_mulheres_lideranca_por_ano['ano_pesquisa'] == ano]
    if not gestoras_ano.empty:
        mulheres_gestoras = gestoras_ano['percentual_mulheres_gestoras'].values[0]
        print(f"Ano {ano}: {total_mulheres:.2f}% de mulheres no total, {mulheres_gestoras:.2f}% das gestoras são mulheres")
    else:
        print(f"Ano {ano}: {total_mulheres:.2f}% de mulheres no total, sem dados sobre gestoras")

Percentual geral de mulheres por ano:
  ano_pesquisa  percentual_mulheres
0         2021            19.819820
1         2022            22.848665
2         2023            25.000000
3         2024            25.634518

Percentual de mulheres em cargos de gestão por ano:
  ano_pesquisa  percentual_mulheres_gestoras
0         2021                     27.659574
1         2022                     19.047619
2         2023                     24.731183
3         2024                     21.250000
Ano 2021: 19.82% de mulheres no total, 27.66% das gestoras são mulheres
Ano 2022: 22.85% de mulheres no total, 19.05% das gestoras são mulheres
Ano 2023: 25.00% de mulheres no total, 24.73% das gestoras são mulheres
Ano 2024: 25.63% de mulheres no total, 21.25% das gestoras são mulheres


In [76]:
# Top 5 formações acadêmicas para homens
formacao_homens = df_rj[df_rj['genero'] == 'Masculino']['area_formacao'].value_counts()
top_formacao_homens = formacao_homens.head(5)
perc_formacao_homens = (formacao_homens.head(5) / formacao_homens.sum() * 100).round(1)

# Top 5 formações acadêmicas para mulheres
formacao_mulheres = df_rj[df_rj['genero'] == 'Feminino']['area_formacao'].value_counts()
top_formacao_mulheres = formacao_mulheres.head(5)
perc_formacao_mulheres = (formacao_mulheres.head(5) / formacao_mulheres.sum() * 100).round(1)

# Visualizar resultados
print("Top 5 formações acadêmicas - Homens:")
for i, (area, contagem) in enumerate(top_formacao_homens.items(), 1):
    percentual = perc_formacao_homens[area]
    print(f"{i}. {area}: {contagem} profissionais ({percentual}%)")

print("\nTop 5 formações acadêmicas - Mulheres:")
for i, (area, contagem) in enumerate(top_formacao_mulheres.items(), 1):
    percentual = perc_formacao_mulheres[area]
    print(f"{i}. {area}: {contagem} profissionais ({percentual}%)")


Top 5 formações acadêmicas - Homens:
1. Computação / Engenharia de Software / TI: 377 profissionais (36.5%)
2. Economia / Administração / Finanças / Negócios: 192 profissionais (18.6%)
3. Engenharia (outras): 159 profissionais (15.4%)
4. Estatística / Matemática / Ciências Atuariais: 98 profissionais (9.5%)
5. Outras Engenharias (não incluir engenharia de software ou TI): 61 profissionais (5.9%)

Top 5 formações acadêmicas - Mulheres:
1. Computação / Engenharia de Software / TI: 93 profissionais (28.4%)
2. Engenharia (outras): 51 profissionais (15.5%)
3. Economia / Administração / Finanças / Negócios: 48 profissionais (14.6%)
4. Estatística / Matemática / Ciências Atuariais: 46 profissionais (14.0%)
5. Outras: 26 profissionais (7.9%)


In [77]:
cargos_interesse = [
    'Analista de Dados', 'Cientista de Dados', 'Engenheiro de Dados', 
    'Desenvolvedor', 'Product Manager', 'Professor/Pesquisador'
]

df_cargos = df_rj[df_rj['cargo'].isin(cargos_interesse)]

# Calcular porcentagem de mulheres por cargo e total de profissionais
resultado_cargos = (
    df_cargos.groupby('cargo')['genero']
    .agg(porcentagem_mulheres=lambda x: (x == 'Feminino').mean() * 100, total_profissionais='count')
    .reset_index()
    .sort_values('porcentagem_mulheres', ascending=False)
)

# Exibir resultados formatados
print("Distribuição de gênero por cargo:")
for _, row in resultado_cargos.iterrows():
    print(f"{row['cargo']}: {row['porcentagem_mulheres']:.1f}% mulheres, {100 - row['porcentagem_mulheres']:.1f}% homens (total: {row['total_profissionais']} profissionais)")

# Média geral de mulheres no setor
media_geral = (df_unificado['genero'] == 'Feminino').mean() * 100
print(f"\nMédia geral de mulheres no setor: {media_geral:.1f}%")

# Comparação entre áreas técnicas e acadêmicas
cargos_tecnicos = ['Cientista de Dados', 'Engenheiro de Dados', 'Desenvolvedor']
cargos_academicos = ['Professor/Pesquisador']

perc_mulheres_tecnicos = df_unificado[df_unificado['cargo'].isin(cargos_tecnicos)]['genero'].eq('Feminino').mean() * 100
perc_mulheres_academicos = df_unificado[df_unificado['cargo'].isin(cargos_academicos)]['genero'].eq('Feminino').mean() * 100

print("\nComparação de representatividade feminina:")
print(f"Cargos técnicos: {perc_mulheres_tecnicos:.1f}% mulheres")
print(f"Cargos acadêmicos: {perc_mulheres_academicos:.1f}% mulheres")


Distribuição de gênero por cargo:
Product Manager: 66.7% mulheres, 33.3% homens (total: 12 profissionais)
Analista de Dados: 26.7% mulheres, 73.3% homens (total: 427 profissionais)
Cientista de Dados: 20.1% mulheres, 79.9% homens (total: 219 profissionais)
Professor/Pesquisador: 20.0% mulheres, 80.0% homens (total: 5 profissionais)
Engenheiro de Dados: 17.9% mulheres, 82.1% homens (total: 145 profissionais)
Desenvolvedor: 17.6% mulheres, 82.4% homens (total: 51 profissionais)

Média geral de mulheres no setor: 23.3%

Comparação de representatividade feminina:
Cargos técnicos: nan% mulheres
Cargos acadêmicos: 18.4% mulheres
